# Trabalho 2

## 4.1

### Imports

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree


### Variáveis

In [ ]:
ficheiro = "../../ficheiros/Trabalho 2/Dados_Trabalho_TP2.csv"
colunas_classes = ['Genero', 'Historico_obesidade_familiar', 'FCCAC', 'CCER', 'Fumador', 'MCC', 'CBA', 'TRANS']


# Funções auxiliares

In [ ]:
def titulo(texto: str):
    print(f"\033[21;30;44m{texto}\033[0m")


def etiqueta_e_valor(etiqueta: str, valor: str = ""):
    print(f"\033[0;94m{etiqueta}: \033[1;94m{valor}\033[0m")


def texto(texto: str, cor="94"):
    print(f"\033[0;{cor}m{texto}\033[0m")


def undersplit(texto):
    return " ".join(texto.split("_"))


def histograma(
        dados: pd.Series,
        grupos: int = 20,
        kde: bool = True,
        size_x: int = 4,
        size_y: int = 4,
        titulo: str = "",
        etiqueta_x: str = "",
        etiqueta_y: str = "",
        color: str = "skyblue",
        edgecolor: str = "black",
):
    """Gera um histograma de forma isolada.
    
    A dimensão final do gráfico é controlada pela relação entre os valores size_x e size_y.
    """
    hist_fig, hist_axes = plt.subplots(figsize=(size_x, size_y))
    sns.histplot(dados, bins=grupos, kde=kde, color=color, edgecolor=edgecolor)
    hist_axes.set_title(titulo)
    hist_axes.set_xlabel(etiqueta_x)
    hist_axes.set_ylabel(etiqueta_y)
    plt.tight_layout()


def tarte(
        dados: pd.Series,
        size_x: int = 4,
        size_y: int = 4,
        autopct="%.2f%%",
        titulo: str = "",
        **kwargs,
):
    pie_fig, pie_axes = plt.subplots(figsize=(size_x, size_y))
    pie_axes.pie(
        dados,
        autopct=autopct,
        **kwargs
    )
    pie_fig.suptitle(titulo)
    return pie_fig, pie_axes


### Inicializações

In [ ]:
warnings.filterwarnings("ignore")  # Desabilitar warnbings.
plt.style.use(
    "style/estilo.mplstyle")  # Garantir que se utiliza um estilo definido centralmente e comum a todos os gráficos.
label_encoder = LabelEncoder()

### 4.1.1

#### Leitura de dados

Como os dados lidos não são em grande quantidade, vamos criar uma cópia do **dataset** para trabalharmos nesta, para poder ter acesso aos "originais" sem ter de os ler novamente do ficheiro.
Por exemplo em alguns dos gráficos, a utilização das *classes* (valores originais) como *etiqueta* é mais elucidativa que usar os valores codificados.  

Na análise ao **dataset** verifica-se que a primeira coluna, identificada como "Unnamed" tem o indíce de cada registo de dados, algo que nos é fornecido pelo **pandas**, e portanto podemos eliminar esta coluna. 
Esta operação estaria incluída nas tarefas do ponto **4.1.4**, mas realizar neste ponto facilita a criação dos gráficos no próximo ponto.

In [ ]:
dados_lidos = pd.read_csv(ficheiro)
dados_trabalho = dados_lidos.copy()
dados_trabalho = dados_trabalho.drop(columns=dados_lidos.columns[0], axis=1)

### Dimensão dos dados

In [ ]:
nr_linhas, nr_colunas = dados_trabalho.shape
titulo("Dimensão dos dados")
etiqueta_e_valor("Número de linhas:", nr_linhas)
etiqueta_e_valor("Número de colunas:", nr_colunas)

### Análise do dataset

#### Atributos

Descrição dos atributos, segundo a documentação. 

| Sigla | Descrição                                                         |
|-------|-------------------------------------------------------------------|
| FCCAC | Frequência de Consumo de Comida Altamente Calórica                | 
| FCV   | Frequência de Consumo de Vegetais                                 | 
| NRP   | Número de Refeições Principais                                    |
| CCER  | Consumo de Comida Entre Refeições                                 |
| CA    | Consumo de Água                                                   |
| CBA   | Consumo de Bebidas Alcoólicas                                     |
| MCC   | Monitorização do Consumo Calorias Histórico de Obesidade Familiar | 

#### Informação sobre o dataset

A função ``info()`` apresenta a informação das colunas (atríbutos): Index da coluna, nome (se tiver, quantidade de registos *não nulos* e o tipo de dados de cada um deles.

In [ ]:
dados_trabalho.info()

#### Amostra do dataset

Para uma rápida visualização dos dados, temos as funções ``head(n)`` e ``tail(n)`` que nos dão *n* linhas, por defeito 5, respetivamente do início ou fim do **dataset**.

In [ ]:
dados_trabalho.head()

In [ ]:
dados_trabalho.tail()

#### Descrição do dados do dateset

A função ``describe()`` analisa os dados do **dataset** e retorna um conjunto de estatísticas descritivas.

In [ ]:
dados_trabalho.describe(include="all")

#### Identificar as classes da coluna Label

In [ ]:
titulo("Classes da coluna \033[1mLabel\033[0m")
for classe in dados_trabalho['Label'].unique():
    texto(classe)

### Identificar as classes da coluna CCER

In [ ]:
titulo("Classes da coluna \033[1mCCER\033[0m")
for classe in dados_trabalho['CCER'].unique():
    texto(classe)

#### identificar classes da coluna TRANS

In [ ]:
titulo("Classes da coluna \033[1mTRANS\033[0m")
for classe in dados_trabalho['TRANS'].unique():
    texto(classe)


### 4.1.2



#### Derivar atributo (IMC)

Depois de criar o novo atributo, podemos usar qualquer uma das funções que foram utilizadas para analisar o **dataset**, neste caso tratando-se de valores numéricos consideramos que a que a mais útil é a ``describe()``.

Podemos também remover as colunas utilizadas para este cálculo uma vez que não vão ser necessárias.

In [ ]:
dados_trabalho['IMC'] = dados_trabalho['Peso'] / (dados_trabalho['Altura'] * dados_trabalho['Altura'])
dados_trabalho.drop(columns=['Peso', 'Altura'], inplace=True)


#### Rever o **dataset** após as alterações

In [ ]:
dados_trabalho.describe(include="all")

Finalmente, usamos novamente a função ``info()`` para confirmar as *series* com que vamos trabalhar de seguida. 

In [ ]:
dados_trabalho.info()

### 4.1.3 Analisar os atributos do conjunto de dados mais significativos, usando gráficos, ...


#### Histogramas

Com estes gráficos podemos ver a distribuição dos valores por "segmentos" de valores.

In [ ]:
sns.set(style="whitegrid")
hist_columns_plot = ['FCV', 'NRP', 'CA', 'FAF', 'TUDE', 'IMC']

for idx, etiqueta_serie in enumerate(hist_columns_plot):
    histograma(
        dados=dados_trabalho[etiqueta_serie],
        grupos=20,
        kde=True,
        size_x=10,
        size_y=4,
        titulo=f"Distribuição de {etiqueta_serie}",
        etiqueta_x=etiqueta_serie,
        etiqueta_y="Frequência",
        color="skyblue",
        edgecolor="black",
    )


#### Pie Charts

A função ``tarte()`` retorna a ``Fig`` e o ``Axes`` para se poder personalizar mais o gráfico.
É também possível passar parâmetros adicionais (*kwargs*), os quais serão usados na invocação de ``pie()``. 

In [ ]:
tarte(
    dados=dados_trabalho["Historico_obesidade_familiar"].value_counts(),
    size_x=4,
    size_y=4,
    titulo="Histórico obesidade familiar",
    labels=dados_lidos["Historico_obesidade_familiar"].unique(),
)

tarte(
    dados=dados_trabalho["FCCAC"].value_counts(),
    size_x=5,
    size_y=5,
    titulo="Frequência de Consumo de Comida Altamente Calórica",
    labels=dados_lidos["FCCAC"].unique(),
)

tarte(
    dados=dados_trabalho["Fumador"].value_counts(),
    size_x=5,
    size_y=5,
    titulo="Fumador",
    labels=dados_lidos["Fumador"].unique(),
)

tarte(
    dados=dados_trabalho["MCC"].value_counts(),
    size_x=5,
    size_y=5,
    titulo="Monitorização do Consumo Calorias",
    labels=dados_lidos["MCC"].unique(),
)

tarte(
    dados=dados_trabalho["CBA"].value_counts(),
    size_x=5,
    size_y=5,
    titulo="Consumo de Bebidas Alcoólicas",
    labels=dados_lidos["CBA"].unique(),
)

tarte(
    dados=dados_trabalho["TRANS"].value_counts(),
    size_x=5,
    size_y=5,
    titulo="Transporte",
    labels=[undersplit(texto) for texto in dados_lidos["TRANS"].unique()],
)

tarte(
    dados=dados_trabalho["Label"].value_counts(),
    size_x=5,
    size_y=5,
    titulo="Classificação do peso",
    labels=[undersplit(texto) for texto in dados_lidos["Label"].unique()],
)

#### visualização de outliers

In [ ]:
boxplots_colunas = ["IMC", "Idade"]

for coluna in boxplots_colunas:
    if (dados_trabalho[coluna].dtype != 'object'):
        dados_trabalho.boxplot(
            column=[coluna],
            by="Label",
            grid=False,
            figsize=(8, 6),
            return_type="dict",
            rot=90,
        )
        plt.title(coluna)

plt.show()

#### Scatterplots

TODO: Fazer scatterplot com relação entre cada um dos preditores e o alvo.

### 4.1.4

#### a) Identificação de NAN e "limpar" o dataset.

Verificar se existem valores nulos.

In [ ]:
dados_trabalho[dados_trabalho.isnull().any(axis=1)]

Verificar se existem valores NaN

In [ ]:
dados_trabalho[dados_trabalho.isna().any(axis=1)]

#### Transformar classes em valores

No mesmo passo em que fazemos a conversão, mostramos os valores finais. 

In [ ]:
for coluna in colunas_classes:
    dados_trabalho[coluna] = label_encoder.fit_transform(dados_trabalho[coluna].values)
    etiqueta_e_valor(coluna, dados_trabalho[coluna].unique())

#### Notas sobre o pré-processamento dos dados

Não foram encontrados valores nulos, nem valor NaN, não sendo por isso necessário fazer tratamento adicionais ao dados neste contexto. 

TODO: Comentário sobre dados inconsistentes e outliers
TODO: Selecionar atributos (?)
TODO: Normalização (?)

### 4.1.5 Matriz de Correlação

In [ ]:
dados_corr = dados_trabalho.copy()
fig_corr = plt.figure(figsize=(12, 8))
etiquetas_corr = [undersplit(texto) for texto in dados_corr.columns]

for coluna in dados_trabalho.columns:
    dados_corr[coluna] = label_encoder.fit_transform(dados_corr[coluna].values)

sns.heatmap(dados_corr.corr(method="spearman").abs(), cmap='coolwarm', annot=True, fmt='.2f', mask=False)

plt.xticks(ticks=[i + 0.5 for i in range(len(dados_corr.columns))], labels=etiquetas_corr, rotation=45, ha='right')
plt.yticks(ticks=[i + 0.5 for i in range(len(dados_corr.columns))], labels=etiquetas_corr)
plt.title('Matriz de Correlação')
plt.tight_layout()

# plt.show()

Por este diagrama podemos ver que não existe uma forte correlação entre nenhum dos atributos. 

### 4.1.6

#### Hold-out

**Hold-out** é o processo de dividir um conjunto de dados em 2 sub-conjuntos de forma aleatória, um de maior dimensão que será utilizado para treinar um algoritmo de Machine Learning e o outra para o testar. 

Predictor: Idade
Target: IMC


TODO: Utilizar os dados que o Pedro enviou e Trabalho 1 como suporte. Mas fazer o modelo sobre 80% dos valores e usar os outros 20% para o cálculo ddo MAE e RMSE

Para o ponto d)



In [ ]:
y = dados_trabalho.IMC
X = dados_trabalho[["Idade"]].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=100)


#### Apresentar informação sobre os datasets de treino e aprendizagem

In [ ]:
titulo("|\t\t\tDimensão\t\t\t|")
titulo("|\tTreino\t\t|\tTeste\t\t|")
texto(f"|\t{X_train.shape}\t|\t{X_test.shape}\t|")
texto(f"|\t{y_train.shape}\t\t|\t{y_test.shape}\t\t|")


#### Inicialização do algoritmo

In [ ]:
regr_lin = LinearRegression()

#### Ajustar os dados

In [ ]:
regr_lin.fit(X_train, y_train)

#### 4.1.6 a)

In [ ]:
etiqueta_e_valor("Equação da reta:", f" y = {regr_lin.intercept_:.3f} + {regr_lin.coef_[0]:.3f} x")

#### 4.1.6 b)

In [ ]:
sns.regplot(x="Idade", y="IMC", data=dados_trabalho, order=1, ci=None, scatter_kws={'color': 'r', 's': 9})
plt.xlim(10, 65)
plt.ylim(ymin=0);

Aplicar o modelo sobre o conjunto de teste.

In [ ]:
y_pred = regr_lin.predict(X_test)


#### MAE

$$MAE = \sum_{i=1}^{D}|x_i-y_i|$$

In [ ]:
MAE = np.mean(abs(y_test - y_pred))
etiqueta_e_valor("MAE no conjunto de dados de teste:", f"{MAE:.3f}")

#### RMSE

$$RMSE = \sqrt{\frac{\sum_{i=0}^{N - 1} (y_i - \hat{y}_i)^2}{N}}$$

In [ ]:
RMSE = np.sqrt(np.mean((y_test - y_pred) ** 2))
etiqueta_e_valor("RMSE no conjunto de dados de teste:", f"{RMSE:.3f}")

#### Repetir o cálculo mas com outros previsores

O preditor vai mudando, podemos ter uma lista com o nome das colunas e fazer um ciclo que realiza os cálculos e guardamos o resultado num dicionário

In [ ]:
# O target não vai mudar.
y = dados_trabalho.IMC

# O preditor vai mudando, podemos ter uma lista com o nome das colunas
lista_preditores = ['Genero', 'Historico_obesidade_familiar', 'FCCAC', 'FCV', 'NRP', 'CCER', 'Fumador', 'CA',
                    'MCC', 'FAF', 'TUDE', 'CBA', 'TRANS']
# Falta o Label


# Resultados
resultados = {}

for preditor in lista_preditores:
    X_preditor = dados_trabalho[[preditor]].to_numpy()
    X_train_1, X_test_1, y_train, y_test = train_test_split(X_preditor, y, train_size=0.8, test_size=0.2, random_state=100)
    line_regr = LinearRegression()
    modelo = line_regr.fit(X_train_1, y_train)
    
    y_pred_1 =  line_regr.predict(X_test_1)
    
    resultados[preditor] = {
        # "X_train": X_train_1,
        # "X_test": X_test_1,
        # "y_train": y_train,
        # "y_test": y_test,
        "modelo": modelo,
        "y_pred": y_pred_1,
        "MAE": np.mean(abs(y_test - y_pred_1)),
        "RMSE": np.sqrt(np.mean((y_test - y_pred_1) ** 2))
    }


#### Tabela com os resultados

Uma vez que temos o pandas, podemos utilizar este para apresentar a informação em formato de tabela.


TODO: Agora é preciso analisar os resultados e talvez produzir algum gráfico ou análise estatistica.

In [ ]:
resultado = pd.DataFrame(resultados)

resultado.loc[["MAE", "RMSE"]]


## 4.2 (?)

#### Separar os dados em target(y) e feature(x)

In [ ]:
y = dados_trabalho.loc[:, "Label"].values
X = dados_trabalho.drop(["Label"], axis=1).values

#labelencoder no y
# le = LabelEncoder()
y = label_encoder.fit_transform(y)

#divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
X_train.shape, X_test.shape

### Árvore de decisão

In [ ]:
clf = DecisionTreeClassifier(random_state=42)
#build the model with training sets
clf.fit(X_train, y_train)

In [ ]:
%matplotlib inline
plt.style.use('ggplot')

plt.figure(figsize=(32, 25))  # set plot size (denoted in inches)
plot_tree(clf,
          feature_names=list(dados_trabalho.columns),
          class_names=list(dados_trabalho['Label']),
          filled=True,
          fontsize=6);

In [ ]:
#verificar a precisão dos dados de treino e teste
clf.score(X_train, y_train)

In [ ]:
# Previsões com decision tree
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

# Avaliação dos dados de treino e teste
tree_train = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", tree_train)
tree_test = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", tree_test)

In [ ]:
# Rate de erro da decision three
cmatrix = confusion_matrix(y_true=y_train, y_pred=clf.predict(X_train), labels=[True, False])
print("Confusion Matrix Training")
print(cmatrix)

# Error rate dos dados de treino
error_rate = (cmatrix[0, 1] + cmatrix[1, 0]) / cmatrix.sum()
print("Training Error Rate:", error_rate)
print("---------------------------------")
ypred = clf.predict(X_test)
cmatrix = confusion_matrix(y_true=y_test, y_pred=ypred, labels=[True, False])
print("Confusion Matrix Test")
print(cmatrix)

# Error rate dos dados de teste
error_rate_test = (cmatrix[0, 1] + cmatrix[1, 0]) / cmatrix.sum()
print("Test Error Rate:", error_rate_test)

In [ ]:
# Matriz de confusão 
def matriz_confusao(actual, predicted):
    # outcome values order in sklearn
    matrix = confusion_matrix(y_true=actual, y_pred=predicted, labels=[True, False])
    disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=[True, False])
    disp.plot()

    # classification report for precision, recall f1-score and accuracy
    matrix = classification_report(actual, predicted)
    print('Classification report Decision Tree: \n', matrix)


# chamada à função
res = matriz_confusao(y_test, ypred)

In [ ]:
# K-Fold Cross Validation - implementation
# Splitting the Data into Folds

def kfold_indices(data, k):
    fold_size = len(data) // k
    indices = np.arange(len(data))
    folds = []
    for i in range(k):
        test_indices = indices[i * fold_size: (i + 1) * fold_size]
        train_indices = np.concatenate([indices[:i * fold_size], indices[(i + 1) * fold_size:]])
        folds.append((train_indices, test_indices))
    return folds


# Define the number of folds (K)
k = 5

# Get the fold indices
fold_indices = kfold_indices(X, k)

In [ ]:
model = DecisionTreeClassifier()

scores = []
prevs_folds = []
y_folds = []
# Iterate through each fold
for train_indices, test_indices in fold_indices:
    X_train, y_train = X[train_indices], y[train_indices]
    X_test, y_test = X[test_indices], y[test_indices]

    # Train the model on the training data
    clf.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = clf.predict(X_test)

    # Calculate the accuracy score for this fold
    fold_score = accuracy_score(y_test, y_pred)

    # Append the fold score to the list of scores
    scores.append(fold_score)

    # Append the prevs and labels of the test set
    prevs_folds.append(y_pred)
    y_folds.append(y_test)

# Calculate the mean accuracy across all folds
mean_accuracy = np.mean(scores)
std_accuracy = np.std(scores)
print("K-Fold Cross-Validation Scores:", scores)
print("Mean Accuracy:", mean_accuracy)
print("Standart Deviation:", std_accuracy)

In [ ]:
# Reconstruir a arvore com diferentes valores
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
import numpy as np

max_depths = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# For each of these values, we want to run the full model cascade from start to finish. 
# We also want to record the train and test scores. We do this in a for loop:
train_score = []
test_score = []
for d in max_depths:
    clf = DecisionTreeClassifier(max_depth=d, random_state=42)
    clf.fit(X_train, y_train)
    train_score.append(clf.score(X_train, y_train))
    test_score.append(clf.score(X_test, y_test))

plt.style.use('ggplot')
plt.figure(figsize=(10, 6))
plt.plot(max_depths, train_score, 'o-', linewidth=3, label='train')
plt.plot(max_depths, test_score, 's-', linewidth=3, label='test')
plt.xlabel('max_depth')
plt.ylabel('score')
plt.ylim(0.2, 1.1)
plt.legend();

In [ ]:
# What about the minimum numbers of samples required to make a node a leaf node? (another Hyperparameter)

train_score = []
test_score = []
min_samples = np.array([2, 4, 8, 16, 32])
for s in min_samples:
    clf = DecisionTreeClassifier(min_samples_leaf=s, random_state=42)
    clf.fit(X_train, y_train)
    train_score.append(clf.score(X_train, y_train))
    test_score.append(clf.score(X_test, y_test))
plt.figure(figsize=(10, 6))
plt.plot(min_samples, train_score, 'o-', linewidth=3, label='train')
plt.plot(min_samples, test_score, 's-', linewidth=3, label='test')
plt.xlabel('min_samples_leaf')
plt.ylabel('score')
plt.ylim(0.7, 1)
plt.legend()